Technical Q&A Assistant

Multi-model chatbot with streaming responses, voice I/O, and function calling.
Supports GPT-4o-mini and Gemini Flash with adaptive expertise levels.

In [ ]:
import os
import json
import tempfile
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import google.generativeai as genai

load_dotenv(override=True)

openai_client = OpenAI()
google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    genai.configure(api_key=google_api_key)

In [ ]:
# Expertise-based system prompts that shape response style
SYSTEM_PROMPTS = {
    "Beginner": """You are a patient and clear technical instructor. 
Explain concepts as if teaching someone new to programming. Use simple analogies, 
avoid jargon when possible, and provide step-by-step explanations.""",
    
    "Intermediate": """You are an experienced software engineer and technical mentor.
Provide clear, practical explanations with code examples. Assume the user has 
basic programming knowledge but may need clarification on advanced concepts.""",
    
    "Expert": """You are a senior software architect and technical expert.
Provide deep, comprehensive explanations with advanced insights. Include 
performance considerations, best practices, and architectural patterns. 
Assume strong technical background."""
}

DEFAULT_SYSTEM_PROMPT = SYSTEM_PROMPTS["Intermediate"]

In [ ]:
# Tool implementations - extend these for richer functionality
def explain_code(code_snippet):
    return f"Code analysis requested for: {code_snippet[:50]}..."

def lookup_documentation(topic):
    docs = {
        "python": "Python is a high-level programming language...",
        "async": "Async/await allows concurrent execution...",
        "gradio": "Gradio is a Python library for building UIs...",
    }
    return docs.get(topic.lower(), f"Documentation for {topic} not found in cache.")

In [ ]:
# OpenAI function calling schema
code_explanation_tool = {
    "type": "function",
    "function": {
        "name": "explain_code",
        "description": "Analyze and explain a code snippet in detail. Use this when the user asks about specific code.",
        "parameters": {
            "type": "object",
            "properties": {
                "code_snippet": {
                    "type": "string",
                    "description": "The code snippet to analyze and explain"
                }
            },
            "required": ["code_snippet"],
            "additionalProperties": False
        }
    }
}

documentation_tool = {
    "type": "function",
    "function": {
        "name": "lookup_documentation",
        "description": "Look up technical documentation for a topic. Use this when the user asks about documentation or references.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The technical topic to look up"
                }
            },
            "required": ["topic"],
            "additionalProperties": False
        }
    }
}

TOOLS = [code_explanation_tool, documentation_tool]

In [ ]:
def handle_explain_code(arguments, tool_call_id):
    code = arguments.get('code_snippet')
    result = explain_code(code)
    return {"role": "tool", "content": result, "tool_call_id": tool_call_id}

def handle_lookup_documentation(arguments, tool_call_id):
    topic = arguments.get('topic')
    result = lookup_documentation(topic)
    return {"role": "tool", "content": result, "tool_call_id": tool_call_id}

TOOL_HANDLERS = {
    "explain_code": handle_explain_code,
    "lookup_documentation": handle_lookup_documentation,
}

In [ ]:
def transcribe_audio(audio_file):
    """Convert speech to text using Whisper."""
    if audio_file is None:
        return None
    try:
        with open(audio_file, "rb") as audio:
            transcript = openai_client.audio.transcriptions.create(
                model="whisper-1",
                file=audio
            )
        return transcript.text
    except Exception as e:
        return f"Error transcribing audio: {str(e)}"


def text_to_speech(text):
    """Generate audio from text using OpenAI TTS."""
    try:
        response = openai_client.audio.speech.create(
            model="tts-1",
            voice="onyx",
            input=text
        )
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        temp_file.write(response.content)
        temp_file.close()
        return temp_file.name
    except Exception as e:
        return None

In [ ]:
def chat_with_openai(message, history, system_prompt, use_tools=True):
    """Stream responses from GPT-4o-mini with optional tool calling."""
    messages = [{"role": "system", "content": system_prompt}]
    
    for msg in history:
        messages.append({"role": msg["role"], "content": msg["content"]})
    
    messages.append({"role": "user", "content": message})
    
    tools = TOOLS if use_tools else None
    stream = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        stream=True
    )
    
    full_response = ""
    tool_calls = []
    
    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            yield content
        
        # Accumulate tool call fragments from streaming response
        if chunk.choices[0].delta.tool_calls:
            for tool_call in chunk.choices[0].delta.tool_calls:
                if tool_call.index is not None:
                    while len(tool_calls) <= tool_call.index:
                        tool_calls.append({"id": "", "function": {"name": "", "arguments": ""}})
                    
                    if tool_call.id:
                        tool_calls[tool_call.index]["id"] = tool_call.id
                    if tool_call.function.name:
                        tool_calls[tool_call.index]["function"]["name"] = tool_call.function.name
                    if tool_call.function.arguments:
                        tool_calls[tool_call.index]["function"]["arguments"] += tool_call.function.arguments
    
    # Execute tools and continue conversation with results
    if tool_calls and use_tools:
        tool_messages = []
        for tool_call in tool_calls:
            if tool_call["function"]["name"]:
                handler = TOOL_HANDLERS.get(tool_call["function"]["name"])
                if handler:
                    arguments = json.loads(tool_call["function"]["arguments"])
                    tool_response = handler(arguments, tool_call["id"])
                    tool_messages.append(tool_response)
        
        messages.append({
            "role": "assistant",
            "content": full_response,
            "tool_calls": [{"id": tc["id"], "function": {"name": tc["function"]["name"], "arguments": tc["function"]["arguments"]}} for tc in tool_calls]
        })
        messages.extend(tool_messages)
        
        final_stream = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            stream=True
        )
        
        for chunk in final_stream:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_response += content
                yield content
    
    return full_response

In [ ]:
def chat_with_gemini(message, history, system_prompt):
    """Stream responses from Gemini Flash."""
    if not google_api_key:
        yield "Error: GOOGLE_API_KEY not configured. Please set it in your .env file."
        return
    
    try:
        model = genai.GenerativeModel(
            model_name="gemini-2.0-flash-exp",
            system_instruction=system_prompt
        )
        
        # Convert history format for Gemini API
        gemini_history = []
        for msg in history:
            if msg["role"] == "user":
                gemini_history.append({"role": "user", "parts": [msg["content"]]})
            elif msg["role"] == "assistant":
                gemini_history.append({"role": "model", "parts": [msg["content"]]})
        
        chat_session = model.start_chat(history=gemini_history)
        response = chat_session.send_message(message, stream=True)
        
        for chunk in response:
            if hasattr(chunk, 'text') and chunk.text:
                yield chunk.text
    except Exception as e:
        yield f"Error with Gemini: {str(e)}"

In [ ]:
def chat(message, history, model_choice, expertise_level, use_tools, audio_enabled):
    """Main chat handler - routes to appropriate model(s)."""
    if not message:
        return history, None
    
    history = history or []
    system_prompt = SYSTEM_PROMPTS.get(expertise_level, DEFAULT_SYSTEM_PROMPT)
    history.append({"role": "user", "content": message})
    
    assistant_response = ""
    
    if model_choice == "GPT-4o-mini" or model_choice == "Both":
        response_text = ""
        for chunk in chat_with_openai(message, history[:-1], system_prompt, use_tools):
            response_text += chunk
            temp_history = history + [{"role": "assistant", "content": response_text}]
            yield temp_history, None
        
        assistant_response = response_text
        
        if model_choice == "Both":
            assistant_response += "\n\n--- Gemini Response ---\n\n"
            gemini_response = ""
            for chunk in chat_with_gemini(message, history[:-1], system_prompt):
                gemini_response += chunk
                combined_response = assistant_response + gemini_response
                temp_history = history + [{"role": "assistant", "content": combined_response}]
                yield temp_history, None
            assistant_response += gemini_response
            history.append({"role": "assistant", "content": assistant_response})
            yield history, None
        else:
            history.append({"role": "assistant", "content": assistant_response})
            yield history, None
    
    elif model_choice == "Gemini Flash":
        response_text = ""
        for chunk in chat_with_gemini(message, history[:-1], system_prompt):
            response_text += chunk
            temp_history = history + [{"role": "assistant", "content": response_text}]
            yield temp_history, None
        
        assistant_response = response_text
        history.append({"role": "assistant", "content": assistant_response})
        yield history, None
    
    audio_file = None
    if audio_enabled and assistant_response:
        audio_file = text_to_speech(assistant_response)
    
    yield history, audio_file

In [ ]:
def process_audio_input(audio_file, history, model_choice, expertise_level, use_tools, audio_enabled):
    """Handle voice input - transcribe and route to chat."""
    if audio_file is None:
        return history, None, None
    
    transcribed_text = transcribe_audio(audio_file)
    
    if not transcribed_text or transcribed_text.startswith("Error"):
        return history, None, None
    
    # Voice input automatically enables audio response
    for hist, audio in chat(transcribed_text, history, model_choice, expertise_level, use_tools, True):
        yield hist, audio, None

In [ ]:
# Build the Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="Technical Question Answerer") as demo:
    gr.Markdown("""
    # 🚀 Technical Q&A Assistant
    
    Ask programming questions and get explanations tailored to your level.
    
    **Features:** Multi-model support • Real-time streaming • Voice I/O • Tool calling
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="Technical Q&A Chat",
                type="messages",
                height=500,
                show_copy_button=True
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    label="Ask a technical question",
                    placeholder="e.g., Explain async/await in Python...",
                    scale=4
                )
                submit_btn = gr.Button("Send 📤", scale=1, variant="primary")
            
            with gr.Row():
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="🎤 Speak your question",
                    show_label=True
                )
        
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Settings")
            
            model_choice = gr.Radio(
                choices=["GPT-4o-mini", "Gemini Flash", "Both"],
                value="GPT-4o-mini",
                label="🤖 Model Selection",
                info="Choose which model(s) to use"
            )
            
            expertise_level = gr.Radio(
                choices=["Beginner", "Intermediate", "Expert"],
                value="Intermediate",
                label="📚 Expertise Level",
                info="Adjust explanation depth"
            )
            
            use_tools = gr.Checkbox(
                value=True,
                label="🔧 Enable Tools",
                info="Use code explanation and documentation tools"
            )
            
            audio_enabled = gr.Checkbox(
                value=True,
                label="🔊 Audio Output",
                info="Generate speech responses"
            )
            
            audio_output = gr.Audio(
                label="🔊 Assistant's Voice Response",
                autoplay=True,
                visible=True
            )
            
            clear_btn = gr.Button("🗑️ Clear Chat", variant="stop")
    
    def toggle_audio_visibility(audio_enabled_val):
        return gr.Audio(visible=audio_enabled_val)
    
    audio_enabled.change(
        fn=toggle_audio_visibility,
        inputs=audio_enabled,
        outputs=audio_output
    )
    
    msg.submit(
        fn=chat,
        inputs=[msg, chatbot, model_choice, expertise_level, use_tools, audio_enabled],
        outputs=[chatbot, audio_output]
    ).then(lambda: "", None, msg)
    
    submit_btn.click(
        fn=chat,
        inputs=[msg, chatbot, model_choice, expertise_level, use_tools, audio_enabled],
        outputs=[chatbot, audio_output]
    ).then(lambda: "", None, msg)
    
    audio_input.stop_recording(
        fn=process_audio_input,
        inputs=[audio_input, chatbot, model_choice, expertise_level, use_tools, audio_enabled],
        outputs=[chatbot, audio_output, audio_input]
    )
    
    clear_btn.click(
        fn=lambda: ([], None, None),
        inputs=None,
        outputs=[chatbot, audio_output, audio_input]
    )
    
    gr.Markdown("""
    ---
    **Tips:** Use the microphone for voice queries • Adjust expertise for simpler/deeper explanations • Enable tools for code analysis
    """)

In [ ]:
demo.launch(share=False, inbrowser=True)